In [1]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=bc18f3ab63ba7d989f48078a536e857cdf2b0fea9075a85813de50f434fdaae3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
POS_TAGS = [
    "adj",
    "adv",
    "intj",
    "noun",
    "propn",
    "verb",
    "adp",
    "aux",
    "cconj",
    "det",
    "num",
    "part",
    "pron",
    "sconj",
    "x"
]

In [36]:
import csv
import pymorphy2

morph = pymorphy2.MorphAnalyzer()


class SearchEngineCSV:
    def __init__(self, csv_file):
        self.sentences = []
        self.tokens = []
        self.lemmas = []
        self.pos_tags = []
        self.chapters = []

        with open(csv_file, newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                self.sentences.append(row[0])
                self.tokens.append([token.lower() for token in row[1].split(';')])  # Токены в нижнем регистре
                self.lemmas.append([lemma.lower() for lemma in row[2].split(';')])  # Леммы в нижнем регистре
                self.pos_tags.append(row[3].split(';'))
                self.chapters.append(row[4])

    def match_token(self, token, word, lemma, pos):
        token = token.lower()  # Приведение токена к нижнему регистру
        if '"' in token:
            return token.strip('"') == word

        if '+' in token:
            query_word, query_pos = token.split('+')
            return lemma == query_word and pos == query_pos

        if token in POS_TAGS:
            return pos.lower() == token

        return lemma == morph.parse(token)[0].normal_form

    def match_sequence(self, query_tokens, words, lemmas, pos_tags):
        if '"' in query_tokens[0]:
            query_tokens[1] = query_tokens[1] + '"'
        for j, query_token in enumerate(query_tokens):
            query_token = query_token.lower()  # Приведение токена запроса к нижнему регистру
            if query_token in POS_TAGS:
                if pos_tags[j].lower() != query_token:
                    return False
            else:
                if not self.match_token(query_token, words[j], lemmas[j], pos_tags[j]):
                    return False
        return True

    def search(self, query):
        query_tokens = query.lower().split()  # Приведение запроса к нижнему регистру

        matches = []
        for idx, sentence in enumerate(self.sentences):
            words = self.tokens[idx]
            lemmas = self.lemmas[idx]
            pos_tags = self.pos_tags[idx]
            if len(words) != len(lemmas) or len(words) != len(pos_tags):  # проверка, что количество токенов, лемм и тегов частей речи одинаковое
                continue

            # Поиск последовательности токенов в предложении
            for i in range(len(words) - len(query_tokens) + 1):
                formatted_sentence = ""
                matches_in_sent = []
                # Проверка последовательности для n-граммы
                if self.match_sequence(query_tokens, words[i:i+len(query_tokens)], lemmas[i:i+len(query_tokens)], pos_tags[i:i+len(query_tokens)]):
                    # Форматируем предложение с источником
                    source_info = f"[{self.chapters[idx]}]"
                    if formatted_sentence == "":
                        formatted_sentence = f"{sentence} {source_info}"
                    matches_in_sent.append((i, i + len(query_tokens)))  # информация о том, какие слова - ответ на запрос (номер первого слова последовательности и первого слова после неё)
                if formatted_sentence != "":
                    matches.append([formatted_sentence, matches_in_sent])  # сначала строка "текст предложения [номер главы]", потом список мест, в которых слова, отвечающие на запрос

        return matches


# Пример использования
csv_file = '/content/corpus_data_with_labels.csv'  # Путь к файлу
search_engine = SearchEngineCSV(csv_file)

# Код для запроса и вывода результатов
print('Введите запрос:')
for_search = input()
results = search_engine.search(for_search)
print(f'Количество примеров: {len(results)}')
set_results = {}
for result in results:
    print(result)

Введите запрос:
ADJ ADJ ADJ
Количество примеров: 20
['— «Ее зовут Бэлою», — отвечал я. И точно, она была хороша: высокая, тоненькая, глаза черные, как у горной серны, так и заглядывали к вам в душу. [Герой нашего времени. Часть 1. Бэла. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-02.html]', [(9, 12)]]
['Говорили про него, что он любяит таскаться за Кубань с абреками\xa07, и, правду сказать, рожа у него была самая разбойничья: маленький, сухой, широкоплечий... А уж ловок-то, ловок-то был, как бес. [Герой нашего времени. Часть 1. Бэла. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-02.html]', [(19, 22)]]
['Говорили про него, что он любяит таскаться за Кубань с абреками\xa07, и, правду сказать, рожа у него была самая разбойничья: маленький, сухой, широкоплечий... А уж ловок-то, ловок-то был, как бес. [Герой нашего времени. Часть 1. Бэла. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-02.html]', [(20, 23)]]
['Говорили про него, что он любяит таскаться за Кубань с абрек

In [ ]:
# Примеры запросов
print(search_engine.search('сказать'))  # Любая форма слова "сказать"
print(search_engine.search('"меня"'))  # Точная форма "меня"
print(search_engine.search('человек+NOUN'))  # Лемма "человек" с POS-тегом "S"
print(search_engine.search('NOUN ADV VERB'))  # Существительное, глагол, наречие в последовательности